Antes de cualquier cosa, debes autentificarte a través del cmd, ejecutando:<br>
earthengine authenticate <br>
y antes deberías instalar pip install earthengine-api geemap geopandas pandas

In [1]:
import geopandas as gpd
import requests
import json
import geopandas as gpd
from shapely.geometry import mapping
import csv
from tqdm import tqdm

Hay que ir a 
https://console.cloud.google.com/apis/dashboard?invt=AbuKWg&project=maximal-osprey-287122
para habilitar la consulta.

In [2]:
# Cargar shapefile local
shapefile_path = "C:/Users/Michael/Documents/ml_zc/ecu_adm_2024/ecu_adm_adm3_2024.shp"
gdf = gpd.read_file(shapefile_path)

In [3]:
parroquias = gdf[['ADM3_PCODE', 'geometry']]
# Solo selecciono dos filas, para reducir la carga de procesamiento

In [4]:
# parroquias = parroquias[parroquias['ADM3_PCODE'].isin(['EC050651', 'EC010951'], )]
# Este polígono no no tiene info en las otras fuentes
parroquias = parroquias[parroquias['ADM3_PCODE'] != 'EC160156']
#parroquias[parroquias['ADM3_PCODE'] == 'EC240352']

In [5]:
# parroquias[parroquias['ADM3_PCODE'] == 'EC180252']
#parroquias = parroquias[1041:]

In [6]:
# Fechas del 31 de diciembre de 2014 a 2024
fechas = [f"{a}-12-31" for a in range(2014, 2025)]

# Endpoint de Ohsome
url = "https://api.ohsome.org/v1/elements/count"

# Ruta a tu archivo
csv_output_path = 'C:/Users/Michael/Documents/ml_zc/datosObtenidos/num_educativos.csv'

# Crear archivo y escribir encabezado
with open(csv_output_path, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=['timestamp', 'educativos', 'ADM3_PCODE'])
    writer.writeheader()

    for num_fila in tqdm(range(parroquias.shape[0]), desc="Procesando parroquias"):
        
        parroquia = parroquias.iloc[num_fila]
        temp_gdf = gpd.GeoDataFrame([parroquia], geometry='geometry', crs=parroquias.crs)
        name_parr = parroquia['ADM3_PCODE']
        print(f"Procesando: {name_parr}")

        # Construir el GeoJSON de la parroquia
        geojson = {
            "type": "FeatureCollection",
            "features": [
                {
                    "type": "Feature",
                    "properties": {"id": name_parr},
                    "geometry": mapping(parroquia['geometry'])
                }
            ]
        }

        # Parámetros para la consulta
        params = {
            "bpolys": json.dumps(geojson),
            "time": ",".join(fechas),
            "filter": "amenity=school",
            "format": "json"
        }

        try:
            res = requests.post(url, data=params)

            if res.status_code == 200:
                data = res.json()
                for entry in data['result']:
                    writer.writerow({
                        "timestamp": entry['timestamp'],
                        "educativos": entry['value'],
                        "ADM3_PCODE": name_parr
                    })
            else:
                print(f"❌ Error {res.status_code} para {name_parr}: {res.text}")

        except Exception as e:
            print(f"⚠️ Error al procesar {name_parr}: {e}")

Procesando parroquias:   0%|                                                                  | 0/1041 [00:00<?, ?it/s]

Procesando: EC010150


Procesando parroquias:   0%|                                                        | 1/1041 [00:04<1:12:45,  4.20s/it]

Procesando: EC010151


Procesando parroquias:   0%|                                                        | 2/1041 [00:08<1:16:30,  4.42s/it]

...

Procesando parroquias: 100%|████████████████████████████████████████████████████▉| 1039/1041 [1:39:17<00:09,  4.61s/it]

Procesando: EC240351


Procesando parroquias: 100%|████████████████████████████████████████████████████▉| 1040/1041 [1:39:20<00:04,  4.25s/it]

Procesando: EC240352


Procesando parroquias: 100%|█████████████████████████████████████████████████████| 1041/1041 [1:39:24<00:00,  5.73s/it]
